# Reading and editing a topography
A topograph/bathymetry is needed to create the domain of interpolation (land-sea mask). The topography/bathymetry is usually extracted from a netCDF file.     
Adapted topographies:
* General Bathymetric Chart of the Oceans ([GEBCO](https://www.gebco.net/))
* [EMODnet Bathymetry](http://www.emodnet-bathymetry.eu)
* Other topographies can be used as long as they define the netCDF variables `lon`, `lat` and `bat` (positive above sea-level and negative below, in `DIVAnd` however, the convention is that `z` is positive in sea water).

This notebook illustrates how the land-sea mask can be modified, for example to exclude a lake.

In [2]:
using DIVAnd

┌ Info: Precompiling DIVAnd [efc8151c-67de-5a8f-9a35-d8f54746ae9d]
└ @ Base loading.jl:1186
ERROR: LoadError: LoadError: UndefVarError: Void not defined
Stacktrace:
 [1] top-level scope at none:0
 [2] include at ./boot.jl:317 [inlined]
 [3] include_relative(::Module, ::String) at ./loading.jl:1038
 [4] include at ./sysimg.jl:29 [inlined]
 [5] include(::String) at /home/ctroupin/.julia/dev/Mustache/src/Mustache.jl:2
 [6] top-level scope at none:0
 [7] include at ./boot.jl:317 [inlined]
 [8] include_relative(::Module, ::String) at ./loading.jl:1038
 [9] include(::Module, ::String) at ./sysimg.jl:29
 [10] top-level scope at none:2
 [11] eval at ./boot.jl:319 [inlined]
 [12] eval(::Expr) at ./client.jl:389
 [13] top-level scope at ./none:3
in expression starting at /home/ctroupin/.julia/dev/Mustache/src/utils.jl:23
in expression starting at /home/ctroupin/.julia/dev/Mustache/src/Mustache.jl:6
ERROR: LoadError: Failed to precompile Mustache [ffc61752-8dc7-55ee-8c37-f3e9cdd09e70] to /home/ct

LoadError: Failed to precompile DIVAnd [efc8151c-67de-5a8f-9a35-d8f54746ae9d] to /home/ctroupin/.julia/compiled/v1.0/DIVAnd/AaVxY.ji.

The full EMODnet Bathymetry is a quite big file (~1GB) that can be obtained from the bathymetry portal: http://portal.emodnet-bathymetry.eu/?menu=19. In the present notebook we will work with the GEBCO bathymetry with a resolution that has been decreased.

Note that an extremely fine bathymetry is not necessary when working on large domains.

## Data download and domain selection
We create a domain encompassing the north of Corsica.

In [3]:
dx = dy = 0.125
lonr = 2.5:dx:12.
latr = 42.3:dy:44.6

# https://b2drop.eudat.eu/s/ACcxUEZZi6a4ziR/download  gebco_30sec_4.nc
# https://b2drop.eudat.eu/s/x83MSV8yUF934bL/download  gebco_30sec_8.nc
# https://b2drop.eudat.eu/s/o0vinoQutAC7eb0/download  gebco_30sec_16.nc

bathname = "gebco_30sec_8.nc"

if !isfile(bathname)
    @info("Download bathymetry")
    download("https://b2drop.eudat.eu/s/x83MSV8yUF934bL/download",bathname)
else
    @info("Bathymetry file already downloaded")
end

bathisglobal = true;

┌ Info: Bathymetry file already downloaded
└ @ Main In[3]:15


We have two main tools to load the bathymetry:
1. `DIVAnd.extract_bath` loads the bathymetry at the resolution defined in the netCDF file while 
2. `DIVAnd.load_bath` reduces the resolution to match the resolution of the analysis.

### Extract bathymetry

In [4]:
?extract_bath

search: extract_bath



```
bx,by,b = DIVAnd.extract_bath(bath_name,isglobal,xi,yi)
```

Extract the bathymetry from the NetCDF file `bathname`. The parameter `isglobal`  is true if the NetCDF file covers the whole globe and thus the last longitude point can be considered to be right next to the first longitude point. `xi` and `yi` are vectors defining the bounding box of the data. No interpolation is performed.

**Convention:** b is positive in the water and negative in the air.


In [6]:
bx,by,b = extract_bath(bathname,true,lonr,latr)
@show size(b);

pcolor(bx,by,b'); colorbar(orientation="horizontal")
gca()[:set_aspect](1/cos(mean([ylim()...]) * pi/180))

size(b) = (147, 39)


┌ Warning: The default `strides(a::AbstractArray)` implementation is deprecated for general arrays.
│ Specialize `strides(::LinearAlgebra.Adjoint)` if `LinearAlgebra.Adjoint` indeed uses a strided representation in memory.
│ Warning: inappropriately implementing this method for an array type that does not use strided
│ storage may lead to incorrect results or segfaults.
│   caller = stride at abstractarray.jl:350 [inlined]
└ @ Core ./abstractarray.jl:350
┌ Warning: The default `strides(a::AbstractArray)` implementation is deprecated for general arrays.
│ Specialize `strides(::LinearAlgebra.Adjoint)` if `LinearAlgebra.Adjoint` indeed uses a strided representation in memory.
│ Warning: inappropriately implementing this method for an array type that does not use strided
│ storage may lead to incorrect results or segfaults.
│   caller = stride at abstractarray.jl:350 [inlined]
└ @ Core ./abstractarray.jl:350
Add `using Statistics` to your imports.
  likely near In[6]:5
┌ Warning: The start

PyCall.PyError: PyError ($(Expr(:escape, :(ccall(#= /home/ctroupin/.julia/packages/PyCall/fiJ3o/src/PyCall.jl:714 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, arg, kw))))) <class 'KeyboardInterrupt'>
KeyboardInterrupt('Julia exception: InterruptException()',)
  File "/home/ctroupin/.julia/packages/Conda/m7vem/deps/usr/lib/python3.6/site-packages/matplotlib/backend_bases.py", line 2263, in print_figure
    **kwargs)
  File "/home/ctroupin/.julia/packages/Conda/m7vem/deps/usr/lib/python3.6/site-packages/matplotlib/backends/backend_svg.py", line 1210, in print_svg
    result = self._print_svg(filename, fh, **kwargs)
  File "/home/ctroupin/.julia/packages/Conda/m7vem/deps/usr/lib/python3.6/site-packages/matplotlib/backends/backend_svg.py", line 1240, in _print_svg
    self.figure.draw(renderer)
  File "/home/ctroupin/.julia/packages/Conda/m7vem/deps/usr/lib/python3.6/site-packages/matplotlib/artist.py", line 55, in draw_wrapper
    return draw(artist, renderer, *args, **kwar

### Load bathymetry
With a change of its original resolution

In [ ]:
?load_bath

In [ ]:
bx,by,b = load_bath(bathname,true,lonr,latr)

pcolor(bx,by,b'); 
colorbar(orientation="horizontal")
gca()[:set_aspect](1/cos(mean([ylim()...]) * pi/180))

#@show size(b)

## Mask creation
We start by defining the mask as a 3D matrix containing only `False`, then we set the values to `True` when the actual depth is deeper than the selected values stored in the vector `depthr`.

In [ ]:
depthr = [0,10,20,500]
mask = falses(size(b,1),size(b,2),length(depthr))

for k = 1:length(depthr)
    for j = 1:size(b,2)
        for i = 1:size(b,1)
            mask[i,j,k] = b[i,j] >= depthr[k]
        end
    end
end

Another solution is to use the `load_mask` function:

In [ ]:
xmask,ymask,mmask = load_mask(bathname,true,lonr,latr,depthr);

In [ ]:
pcolor(bx,by,Float64.(mask[:,:,1])'); 
colorbar(orientation="horizontal")
gca()[:set_aspect](1/cos(mean([ylim()...]) * pi/180)) # fixes the aspect ratio

In [ ]:
pcolor(xmask,ymask,Float64.(mmask[:,:,1])'); 
colorbar(orientation="horizontal")
gca()[:set_aspect](1/cos(mean([ylim()...]) * pi/180)) # fixes the aspect ratio

## Mask editing
To edit the mask, you can either modify directly its value if you know the indices, or you can use a selection based on the coordinates.
### Create an artificial island
We simply modify the mask at a given location.

In [ ]:
# Create a copy of the mask for ediding
mask_copy = copy(mask);
mask_copy[63,8,:] = false

In [ ]:
pcolor(bx, by, Float64.(mask_copy[:,:,1])'); 
colorbar(orientation="horizontal")
gca()[:set_aspect](1/cos(mean([ylim()...]) * pi/180))

### Remove an island (based on indices)

In [ ]:
mask_copy = copy(mask);
mask_copy[48:59,1:10,:] = true

In [ ]:
pcolor(bx, by, Float64.(mask_copy[:,:,1])'); 
colorbar(orientation="horizontal")
gca()[:set_aspect](1/cos(mean([ylim()...]) * pi/180))

### Remove an island (based on coordinates). Recommended approach as you can change resolution easily
`sel_lon` and `sel_lat` are obtained by checking the longitude and latitudes.

In [ ]:
mask_copy = copy(mask);
sel_lon = (bx.>8) .& (bx.<10.);
sel_lat = (by.<43.25);
mask_copy[sel_lon, sel_lat, :]=true;
pcolor(bx, by, Float64.(mask_copy[:,:,1])'); 
colorbar(orientation="horizontal")
gca()[:set_aspect](1/cos(mean([ylim()...]) * pi/180))

In [ ]:
sel_lon

## Exercice
1. Load the bathymetry around the Black Sea.
2. Create a land sea mask for the following levels: 0, 150, 500 and 1000 meters.
3. Remove the Seas of Azov and the Sea of Marmara.
4. Plot the results.

The solution is available in [06-topography-BlackSea](./solutions/06-topography-BlackSea) and should look like this:

<img src="./Images/BlackSea_mask.png" width="250px">